In [ ]:
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
!pip install paddlepaddle

#Converting PDF to images

In [ ]:
!pip install pdf2image
!apt-get update
!apt-get install poppler-utils

In [ ]:
from pdf2image import convert_from_path

In [ ]:
images = convert_from_path('/content/j.ijhydene.2014.06.160.pdf')

In [ ]:
!mkdir pages_1

In [ ]:
total_page = 0
for i in range(len(images)):
  images[i].save('pages_1/page'+str(i)+'.jpg', 'JPEG')
  total_page +=1

In [ ]:
total_page

6

#getting tables

In [ ]:
#!python3 -m pip install paddlepaddle-gpu
!pip install "paddleocr>=2.0.1"
!pip install protobuf
!git clone https://github.com/PaddlePaddle/PaddleOCR.git


In [ ]:
!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

In [ ]:
!wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.0g-2ubuntu4_amd64.deb
!sudo dpkg -i libssl1.1_1.1.0g-2ubuntu4_amd64.deb

In [ ]:
!pip install --upgrade paddleocr
!pip install --upgrade paddlepaddle

In [ ]:
from paddleocr import PaddleOCR, draw_ocr
import cv2
import layoutparser as lp

In [ ]:
# load model
model = lp.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                threshold=0.5,
                                label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
                                enforce_cpu=False,
                                enable_mkldnn=True)#math kernel library

download https://paddle-model-ecology.bj.bcebos.com/model/layout-parser/ppyolov2_r50vd_dcn_365e_publaynet.tar to /root/.paddledet/inference_model/ppyolov2_r50vd_dcn_365e_publaynet/ppyolov2_r50vd_dcn_365e_publaynet_infer/ppyolov2_r50vd_dcn_365e_publaynet.tar


100%|██████████| 221M/221M [00:31<00:00, 7.09MiB/s]


In [ ]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

In [ ]:
def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0

  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

  return inter / float(box_1_area + box_2_area - inter)

In [ ]:
for num in range(total_page):
  image = cv2.imread('/content/pages_1/page'+str(num)+'.jpg')

  image = image[..., ::-1]

  # detect
  layout = model.detect(image)

  counts=0
  for l in layout:
      if l.type == 'Table':
        counts +=1
  if counts!=0:
    x_1=[0]*counts
    y_1=[0]*counts
    x_2=[0]*counts
    y_2=[0]*counts
    i=0
    for l in layout:
      #print(l)
      if l.type == 'Table':
        x_1[i] = int(l.block.x_1)
        #print(l.block.x_1)
        y_1[i] = int(l.block.y_1)
        x_2[i] = int(l.block.x_2)
        y_2[i] = int(l.block.y_2)
        print(x_1[i],y_1[i],x_2[i],y_2[i])
        i+=1


    im = cv2.imread('/content/pages_1/page'+str(num)+'.jpg')
    ocr = PaddleOCR(lang='en')
    for count in range(counts):
      print("Current Page number is",num)
      print("Current Table count is ",count)
      print(x_1[count],y_1[count],x_2[count],y_2[count])
      cv2.imwrite('ext_im'+str(num)+str(count+1)+'.jpg', im[y_1[count]:y_2[count],x_1[count]:x_2[count]])


      image_path = '/content/ext_im'+str(num)+str(count+1)+'.jpg'
      image_cv = cv2.imread(image_path)
      image_height = image_cv.shape[0]
      image_width = image_cv.shape[1]
      output = ocr.ocr(image_path)[0]

      boxes = [line[0] for line in output]
      texts = [line[1][0] for line in output]
      probabilities = [line[1][1] for line in output]

      image_boxes = image_cv.copy()

      for box,text in zip(boxes,texts):
        cv2.rectangle(image_boxes, (int(box[0][0]),int(box[0][1])), (int(box[2][0]),int(box[2][1])),(0,0,255),1)
        cv2.putText(image_boxes, text,(int(box[0][0]),int(box[0][1])),cv2.FONT_HERSHEY_SIMPLEX,1,(222,0,0),1)

      cv2.imwrite('detections'+str(num)+str(count+1)+'.jpg', image_boxes)

    #get horizontal and vertical lines
      imx = image_cv.copy()

      horiz_boxes = []
      vert_boxes = []

      for box in boxes:
        x_h, x_v = 0,int(box[0][0])
        y_h, y_v = int(box[0][1]),0
        width_h,width_v = image_width, int(box[2][0]-box[0][0])
        height_h,height_v = int(box[2][1]-box[0][1]),image_height

        horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
        vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

        cv2.rectangle(imx,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
        cv2.rectangle(imx,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)

      #cv2.imwrite('horiz_vert'+str(num)+str(count+1)+'.jpg',imx)

      #Non Max Suppresion
      horiz_out = tf.image.non_max_suppression(
        horiz_boxes,
        probabilities,
        max_output_size = 1000,
        iou_threshold=0.1,
        score_threshold=float('-inf'),
        name=None
      )
      horiz_lines = np.sort(np.array(horiz_out))
      im_nms = image_cv.copy()
      for val in horiz_lines:
        cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)

      #cv2.imwrite('im_nms_h'+str(num)+str(count+1)+'.jpg',im_nms)

      vert_out = tf.image.non_max_suppression(
        vert_boxes,
        probabilities,
        max_output_size = 1000,
        iou_threshold=0.1,
        score_threshold=float('-inf'),
        name=None
      )
      vert_lines = np.sort(np.array(vert_out))
      im_nms = image_cv.copy()

      for val in vert_lines:
        cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)

      #cv2.imwrite('im_nms_v'+str(num)+str(count+1)+'.jpg',im_nms)

    #Convert to CSV
      out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
      # print(np.array(out_array).shape)
      # print(out_array)

      unordered_boxes = []

      for i in vert_lines:
        #print(vert_boxes[i])
        unordered_boxes.append(vert_boxes[i][0])

      ordered_boxes = np.argsort(unordered_boxes)

      for i in range(len(horiz_lines)):
        for j in range(len(vert_lines)):
          resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

          for b in range(len(boxes)):
            the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
            if(iou(resultant,the_box)>0.1):
              out_array[i][j] = texts[b]

      out_array=np.array(out_array)
      pd.DataFrame(out_array).to_csv('/content/Sheet_'+str(num)+str(count+1)+'.csv')

130 230 1512 417
download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:16<00:00, 249kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10.2M/10.2M [00:15<00:00, 644kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:13<00:00, 157kiB/s]

[2023/08/08 15:29:17] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='

Current Page number is 3
Current Table count is  0
130 230 1512 417
[2023/08/08 15:29:17] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
[2023/08/08 15:29:17] ppocr DEBUG: dt_boxes num : 48, elapse : 0.12746357917785645
[2023/08/08 15:29:25] ppocr DEBUG: rec_res num  : 48, elapse : 7.115644454956055


In [ ]:
counts

0

#New

##Mounting drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##importing libraries

In [ ]:
#general libraries
from PIL import Image
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [ ]:
#For pdf2image
!pip install pdf2image
!apt-get update
!apt-get install poppler-utils
from pdf2image import convert_from_path

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [980 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,136 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [860 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,241 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/g

###paddlepaddle

In [ ]:
!pip install paddlepaddle-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 541.0/541.0 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.2/383.2 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.5 MB/s eta 0:00:00


In [ ]:
#For paddlepaddle
!pip install "paddleocr>=2.0.1"
!pip install protobuf
!git clone https://github.com/PaddlePaddle/PaddleOCR.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.1/466.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 75.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.4/148.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Cloning into 'PaddleOCR'...
remote: Enumerating objects: 47048, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 47048 (delta 169), reused 207 (delta 120), pack-reused 46756
Receiving objects: 100% (47048/47048), 343.32 MiB | 30.29 MiB/s, done.
Resolving deltas: 100% (33019/33019), done.
Updating files: 100% (1889/1889), done.


In [ ]:
!wget https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
!pip install -U layoutparser-0.0.0-py3-none-any.whl

--2023-08-22 11:59:10--  https://paddleocr.bj.bcebos.com/whl/layoutparser-0.0.0-py3-none-any.whl
Resolving paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)... 103.235.46.61, 2409:8c04:1001:1002:0:ff:b001:368a
Connecting to paddleocr.bj.bcebos.com (paddleocr.bj.bcebos.com)|103.235.46.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19145360 (18M) [application/octet-stream]
Saving to: ‘layoutparser-0.0.0-py3-none-any.whl’

layoutparser-0.0.0- 100%[===================>]  18.26M  3.31MB/s    in 17s     

2023-08-22 11:59:28 (1.09 MB/s) - ‘layoutparser-0.0.0-py3-none-any.whl’ saved [19145360/19145360]

Processing ./layoutparser-0.0.0-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 701.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=4e46c708a02388c8e74d38e0160692f9e89767e0cef7787eba5d9f2c7e50609a
  Stored in directory: /root/.cache/pip/whee

In [ ]:
!wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.0g-2ubuntu4_amd64.deb
!sudo dpkg -i libssl1.1_1.1.0g-2ubuntu4_amd64.deb
!pip install --upgrade paddleocr
!pip install --upgrade paddlepaddle-gpu

In [ ]:
from pdf2image import convert_from_path
from paddleocr import PaddleOCR, draw_ocr
import cv2
import layoutparser as lp

##Execution

In [ ]:
# Paths
pdf_folder = '/content/drive/MyDrive/5th Year (2023)/AmritanshuKhareDualDegree2024/MTP_Dataset_todo'  # Replace with your PDFs folder path
output_folder = '/content/drive/MyDrive/5th Year (2023)/AmritanshuKhareDualDegree2024/Test_Output_6'  # Replace with desired output folder path

In [ ]:
# load model
model = lp.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
                                threshold=0.6,
                                label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
                                enforce_cpu=False,
                                enable_mkldnn=True)#math kernel library

download https://paddle-model-ecology.bj.bcebos.com/model/layout-parser/ppyolov2_r50vd_dcn_365e_publaynet.tar to /root/.paddledet/inference_model/ppyolov2_r50vd_dcn_365e_publaynet/ppyolov2_r50vd_dcn_365e_publaynet_infer/ppyolov2_r50vd_dcn_365e_publaynet.tar


100%|██████████| 221M/221M [00:28<00:00, 7.80MiB/s]


###Processing each pdf file

In [ ]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

In [ ]:
def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0

  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

  return inter / float(box_1_area + box_2_area - inter)

In [ ]:
pdf_files = os.listdir(pdf_folder)
for pdf_file in pdf_files:
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, pdf_file)
        pdf_name = os.path.splitext(pdf_file)[0]
        print(pdf_name)
        # Create a directory for PDF-specific data
        pdf_output_dir = os.path.join(output_folder, pdf_name)
        os.makedirs(pdf_output_dir, exist_ok=True)



        # Convert PDF pages to images
        pdf_images = convert_from_path(pdf_path)
        page_dir = os.path.join(pdf_output_dir, pdf_name + '_pages')
        os.makedirs(page_dir, exist_ok=True)

        total_pages=0

        for page_num, image in enumerate(pdf_images):
            page_image_path = os.path.join(page_dir, f'page{page_num + 1}.jpg')
            image.save(page_image_path, 'JPEG')
            total_pages+=1

            image = cv2.imread(page_image_path)
            image = image[..., ::-1]
            layout = model.detect(image)
            counts=0
            for l in layout:
                if l.type == 'Table':
                  counts +=1
            #print("for page no.", page_num+1," the table_count = ",counts)


            #if tables are present in this page
            if counts!=0:
              #print("counts")

              # Create a directory for extracted table images
              table_img_dir = os.path.join(pdf_output_dir, 'table_imgs')
              os.makedirs(table_img_dir, exist_ok=True)

              x_1=[0]*counts
              y_1=[0]*counts
              x_2=[0]*counts
              y_2=[0]*counts
              i=0
              for l in layout:
                #print(l)
                if l.type == 'Table':
                  x_1[i] = int(l.block.x_1)
                  #print(l.block.x_1)
                  y_1[i] = int(l.block.y_1)
                  x_2[i] = int(l.block.x_2)
                  y_2[i] = int(l.block.y_2)
                  print(x_1[i],y_1[i],x_2[i],y_2[i])
                  i+=1

              ocr = PaddleOCR(lang='en')
              for count in range(counts):
                  # Extract the table image
                table_image = image[y_1[count]:y_2[count], x_1[count]:x_2[count]]

                  # Save the table image with the specified naming format
                table_img_path = os.path.join(table_img_dir, f'table_img_{page_num + 1}_{count + 1}.jpg')
                cv2.imwrite(table_img_path, table_image)
                #print("this ran")


              # Iterate through table images in 'table_imgs' directory
              table_img_dir = os.path.join(pdf_output_dir, 'table_imgs')
              table_images = [f for f in os.listdir(table_img_dir) if f.endswith('.jpg')]

              for table_img_file in table_images:
                  # Skip detection images
                  if table_img_file.startswith('detections_'):
                      continue

                  table_img_path = os.path.join(table_img_dir, table_img_file)
                  table_image_cv = cv2.imread(table_img_path)
                  image_height = table_image_cv.shape[0]
                  image_width = table_image_cv.shape[1]

                  # Perform OCR on the table image
                  output = ocr.ocr(table_img_path)[0]
                  boxes = [line[0] for line in output]
                  texts = [line[1][0] for line in output]
                  probabilities = [line[1][1] for line in output]

                  image_boxes = table_image_cv.copy()

                  for box, text in zip(boxes, texts):
                      cv2.rectangle(image_boxes, (int(box[0][0]), int(box[0][1])), (int(box[2][0]), int(box[2][1])), (0, 0, 255), 1)
                      cv2.putText(image_boxes, text, (int(box[0][0]), int(box[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (222, 0, 0), 1)

                  detections_img_path = os.path.join(table_img_dir, f'detections_{table_img_file}')
                  cv2.imwrite(detections_img_path, image_boxes)

              imx = table_image_cv.copy()

              horiz_boxes = []
              vert_boxes = []

              for box in boxes:
                x_h, x_v = 0,int(box[0][0])
                y_h, y_v = int(box[0][1]),0
                width_h,width_v = image_width, int(box[2][0]-box[0][0])
                height_h,height_v = int(box[2][1]-box[0][1]),image_height

                horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
                vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

                cv2.rectangle(imx,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
                cv2.rectangle(imx,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)

              #cv2.imwrite('horiz_vert'+str(num)+str(count+1)+'.jpg',imx)

              #Non Max Suppresion
              horiz_out = tf.image.non_max_suppression(
                horiz_boxes,
                probabilities,
                max_output_size = 1000,
                iou_threshold=0.1,
                score_threshold=float('-inf'),
                name=None
              )
              horiz_lines = np.sort(np.array(horiz_out))
              im_nms = table_image_cv.copy()
              for val in horiz_lines:
                cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)

              #cv2.imwrite('im_nms_h'+str(num)+str(count+1)+'.jpg',im_nms)

              vert_out = tf.image.non_max_suppression(
                vert_boxes,
                probabilities,
                max_output_size = 1000,
                iou_threshold=0.1,
                score_threshold=float('-inf'),
                name=None
              )
              vert_lines = np.sort(np.array(vert_out))
              im_nms = table_image_cv.copy()

              for val in vert_lines:
                cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)

              #cv2.imwrite('im_nms_v'+str(num)+str(count+1)+'.jpg',im_nms)

            # Create a directory for CSV files
              csv_dir = os.path.join(pdf_output_dir, 'csv')
              os.makedirs(csv_dir, exist_ok=True)

            #Convert to CSV
              out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
              # print(np.array(out_array).shape)
              # print(out_array)

              unordered_boxes = []

              for i in vert_lines:
                #print(vert_boxes[i])
                unordered_boxes.append(vert_boxes[i][0])

              ordered_boxes = np.argsort(unordered_boxes)

              for i in range(len(horiz_lines)):
                for j in range(len(vert_lines)):
                  resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

                  for b in range(len(boxes)):
                    the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
                    if(iou(resultant,the_box)>0.1):
                      out_array[i][j] = texts[b]

              print(counts)
              # Create the CSV file path and name
              for count in range(counts):
                csv_filename = f'{pdf_name}_{page_num}_{count}.csv'
                csv_path = os.path.join(csv_dir, csv_filename)
                out_array = np.array(out_array)
                pd.DataFrame(out_array).to_csv(csv_path)


###Rough

In [ ]:
pdf_files = os.listdir(pdf_folder)
for pdf_file in pdf_files:
    if pdf_file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, pdf_file)
        pdf_name = os.path.splitext(pdf_file)[0]
        print(pdf_name)
        # Create a directory for PDF-specific data
        pdf_output_dir = os.path.join(output_folder, pdf_name)
        os.makedirs(pdf_output_dir, exist_ok=True)



        # Convert PDF pages to images
        pdf_images = convert_from_path(pdf_path)
        page_dir = os.path.join(pdf_output_dir, pdf_name + '_pages')
        os.makedirs(page_dir, exist_ok=True)

        total_pages=0

        for page_num, image in enumerate(pdf_images):
            page_image_path = os.path.join(page_dir, f'page{page_num + 1}.jpg')
            image.save(page_image_path, 'JPEG')
            total_pages+=1

            image = cv2.imread(page_image_path)
            image = image[..., ::-1]
            layout = model.detect(image)
            counts=0
            for l in layout:
                if l.type == 'Table':
                  counts +=1
            #print("for page no.", page_num+1," the table_count = ",counts)


            #if tables are present in this page
            if counts!=0:
              #print("counts")

              # Create a directory for extracted table images
              table_img_dir = os.path.join(pdf_output_dir, 'table_imgs')
              os.makedirs(table_img_dir, exist_ok=True)

              x_1=[0]*counts
              y_1=[0]*counts
              x_2=[0]*counts
              y_2=[0]*counts
              i=0
              for l in layout:
                #print(l)
                if l.type == 'Table':
                  x_1[i] = int(l.block.x_1)
                  #print(l.block.x_1)
                  y_1[i] = int(l.block.y_1)
                  x_2[i] = int(l.block.x_2)
                  y_2[i] = int(l.block.y_2)
                  print(x_1[i],y_1[i],x_2[i],y_2[i])
                  i+=1

In [ ]:
              ocr = PaddleOCR(lang='en')
              for count in range(counts):
                  # Extract the table image
                table_image = image[y_1[count]:y_2[count], x_1[count]:x_2[count]]

                  # Save the table image with the specified naming format
                table_img_path = os.path.join(table_img_dir, f'table_img_{page_num + 1}_{count + 1}.jpg')
                cv2.imwrite(table_img_path, table_image)
                #print("this ran")

In [ ]:
              # Iterate through table images in 'table_imgs' directory
              table_img_dir = os.path.join(pdf_output_dir, 'table_imgs')
              table_images = [f for f in os.listdir(table_img_dir) if f.endswith('.jpg')]

              for table_img_file in table_images:
                  # Skip detection images
                  if table_img_file.startswith('detections_'):
                      continue

                  table_img_path = os.path.join(table_img_dir, table_img_file)
                  table_image_cv = cv2.imread(table_img_path)
                  image_height = table_image_cv.shape[0]
                  image_width = table_image_cv.shape[1]

                  # Perform OCR on the table image
                  output = ocr.ocr(table_img_path)[0]
                  boxes = [line[0] for line in output]
                  texts = [line[1][0] for line in output]
                  probabilities = [line[1][1] for line in output]

                  image_boxes = table_image_cv.copy()

                  for box, text in zip(boxes, texts):
                      cv2.rectangle(image_boxes, (int(box[0][0]), int(box[0][1])), (int(box[2][0]), int(box[2][1])), (0, 0, 255), 1)
                      cv2.putText(image_boxes, text, (int(box[0][0]), int(box[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (222, 0, 0), 1)

                  detections_img_path = os.path.join(table_img_dir, f'detections_{table_img_file}')
                  cv2.imwrite(detections_img_path, image_boxes)


In [ ]:
              imx = table_image_cv.copy()

              horiz_boxes = []
              vert_boxes = []

              for box in boxes:
                x_h, x_v = 0,int(box[0][0])
                y_h, y_v = int(box[0][1]),0
                width_h,width_v = image_width, int(box[2][0]-box[0][0])
                height_h,height_v = int(box[2][1]-box[0][1]),image_height

                horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
                vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

                cv2.rectangle(imx,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
                cv2.rectangle(imx,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)

              #cv2.imwrite('horiz_vert'+str(num)+str(count+1)+'.jpg',imx)


In [ ]:
              #Non Max Suppresion
              horiz_out = tf.image.non_max_suppression(
                horiz_boxes,
                probabilities,
                max_output_size = 1000,
                iou_threshold=0.1,
                score_threshold=float('-inf'),
                name=None
              )
              horiz_lines = np.sort(np.array(horiz_out))
              im_nms = image_cv.copy()
              for val in horiz_lines:
                cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)

              #cv2.imwrite('im_nms_h'+str(num)+str(count+1)+'.jpg',im_nms)

              vert_out = tf.image.non_max_suppression(
                vert_boxes,
                probabilities,
                max_output_size = 1000,
                iou_threshold=0.1,
                score_threshold=float('-inf'),
                name=None
              )
              vert_lines = np.sort(np.array(vert_out))
              im_nms = table_image_cv.copy()

              for val in vert_lines:
                cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)

              #cv2.imwrite('im_nms_v'+str(num)+str(count+1)+'.jpg',im_nms)


#Modular Code

In [ ]:
def intersection(box_1, box_2):
  return [box_2[0], box_1[1],box_2[2], box_1[3]]

In [ ]:
def iou(box_1, box_2):

  x_1 = max(box_1[0], box_2[0])
  y_1 = max(box_1[1], box_2[1])
  x_2 = min(box_1[2], box_2[2])
  y_2 = min(box_1[3], box_2[3])

  inter = abs(max((x_2 - x_1, 0)) * max((y_2 - y_1), 0))
  if inter == 0:
      return 0

  box_1_area = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
  box_2_area = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

  return inter / float(box_1_area + box_2_area - inter)

In [ ]:
def process_table_images(table_images_dir, pdf_name, page_num):
  ocr = PaddleOCR(lang='en')
  table_images = [f for f in os.listdir(table_images_dir) if f.endswith('.jpg')]
  print(table_images)

  table_image_cv = cv2.imread(table_img_path)
  if table_image_cv is None:
    print(f"Error reading image: {table_img_path}")

  # ... (Rest of your code to process table images and create CSV)


  for table_img_file in table_images:
    table_img_path = os.path.join(table_images_dir, table_img_file)
    print("Table Image Path:", table_img_path)

    # Skip detection images
    if table_img_file.startswith('detections_'):
      continue

    table_img_path = os.path.join(table_img_dir, table_img_file)
    table_image_cv = cv2.imread(table_img_path)
    image_height = table_image_cv.shape[0]
    image_width = table_image_cv.shape[1]

    # Perform OCR on the table image
    output = ocr.ocr(table_img_path)[0]
    boxes = [line[0] for line in output]
    texts = [line[1][0] for line in output]
    probabilities = [line[1][1] for line in output]

    image_boxes = table_image_cv.copy()

    for box, text in zip(boxes, texts):
      cv2.rectangle(image_boxes, (int(box[0][0]), int(box[0][1])), (int(box[2][0]), int(box[2][1])), (0, 0, 255), 1)
      cv2.putText(image_boxes, text, (int(box[0][0]), int(box[0][1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (222, 0, 0), 1)

    detections_img_path = os.path.join(table_img_dir, f'detections_{table_img_file}')
    cv2.imwrite(detections_img_path, image_boxes)

  imx = table_image_cv.copy()

  horiz_boxes = []
  vert_boxes = []

  for box in boxes:
    x_h, x_v = 0,int(box[0][0])
    y_h, y_v = int(box[0][1]),0
    width_h,width_v = image_width, int(box[2][0]-box[0][0])
    height_h,height_v = int(box[2][1]-box[0][1]),image_height

    horiz_boxes.append([x_h,y_h,x_h+width_h,y_h+height_h])
    vert_boxes.append([x_v,y_v,x_v+width_v,y_v+height_v])

    cv2.rectangle(imx,(x_h,y_h), (x_h+width_h,y_h+height_h),(0,0,255),1)
    cv2.rectangle(imx,(x_v,y_v), (x_v+width_v,y_v+height_v),(0,255,0),1)

  #cv2.imwrite('horiz_vert'+str(num)+str(count+1)+'.jpg',imx)

  #Non Max Suppresion
  horiz_out = tf.image.non_max_suppression(
    horiz_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
  )
  horiz_lines = np.sort(np.array(horiz_out))
  im_nms = table_image_cv.copy()
  for val in horiz_lines:
    cv2.rectangle(im_nms, (int(horiz_boxes[val][0]),int(horiz_boxes[val][1])), (int(horiz_boxes[val][2]),int(horiz_boxes[val][3])),(0,0,255),1)

  #cv2.imwrite('im_nms_h'+str(num)+str(count+1)+'.jpg',im_nms)

  vert_out = tf.image.non_max_suppression(
    vert_boxes,
    probabilities,
    max_output_size = 1000,
    iou_threshold=0.1,
    score_threshold=float('-inf'),
    name=None
  )
  vert_lines = np.sort(np.array(vert_out))
  im_nms = table_image_cv.copy()

  for val in vert_lines:
    cv2.rectangle(im_nms, (int(vert_boxes[val][0]),int(vert_boxes[val][1])), (int(vert_boxes[val][2]),int(vert_boxes[val][3])),(255,0,0),1)

  #Convert to CSV
  out_array = [["" for i in range(len(vert_lines))] for j in range(len(horiz_lines))]
  # print(np.array(out_array).shape)
  # print(out_array)

  unordered_boxes = []

  for i in vert_lines:
    #print(vert_boxes[i])
    unordered_boxes.append(vert_boxes[i][0])

  ordered_boxes = np.argsort(unordered_boxes)

  for i in range(len(horiz_lines)):
    for j in range(len(vert_lines)):
      resultant = intersection(horiz_boxes[horiz_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]] )

      for b in range(len(boxes)):
        the_box = [boxes[b][0][0],boxes[b][0][1],boxes[b][2][0],boxes[b][2][1]]
        if(iou(resultant,the_box)>0.1):
          out_array[i][j] = texts[b]

  print(counts)
  # Create the CSV file path and name
  for count in range(counts):
    csv_filename = f'{pdf_name}_{page_num}_{count}.csv'
    csv_path = os.path.join(csv_dir, csv_filename)
    out_array = np.array(out_array)
    pd.DataFrame(out_array).to_csv(csv_path)


In [ ]:
def create_table_images(layout, image, table_images_dir,pdf_name,page_num):
# ... (Your code to extract and save table images)
  counts=0
  for l in layout:
      if l.type == 'Table':
        counts +=1
  #print("for page no.", page_num+1," the table_count = ",counts)


  #if tables are present in this page
  if counts!=0:

    x_1=[0]*counts
    y_1=[0]*counts
    x_2=[0]*counts
    y_2=[0]*counts
    i=0
    for l in layout:
      #print(l)
      if l.type == 'Table':
        x_1[i] = int(l.block.x_1)
        #print(l.block.x_1)
        y_1[i] = int(l.block.y_1)
        x_2[i] = int(l.block.x_2)
        y_2[i] = int(l.block.y_2)
        print(x_1[i],y_1[i],x_2[i],y_2[i])
        i+=1

    ocr = PaddleOCR(lang='en')
    print("counts is:",counts)
    for count in range(counts):
      # Extract the table image
      table_image = image[y_1[count]:y_2[count], x_1[count]:x_2[count]]
      print("is this running")
      # Save the table image with the specified naming format
      table_img_path = os.path.join(table_images_dir, f'table_img_{page_num + 1}_{count + 1}.jpg')
      cv2.imwrite(table_img_path, table_image)

    process_table_images(table_images_dir, pdf_name, page_num)

In [ ]:
def process_pdf(pdf_file, pdf_folder, output_folder):
  pdf_name = os.path.splitext(pdf_file)[0]
  print(pdf_name)
  pdf_path = os.path.join(pdf_folder, pdf_file)

  pdf_output_dir = os.path.join(output_folder, pdf_name)
  os.makedirs(pdf_output_dir, exist_ok=True)

  pdf_images = convert_from_path(pdf_path)
  page_dir = os.path.join(pdf_output_dir, pdf_name + '_pages')
  os.makedirs(page_dir, exist_ok=True)

  table_images_dir = os.path.join(pdf_output_dir, 'table_imgs')
  os.makedirs(table_images_dir, exist_ok=True)

  # ... (Rest of the processing code)
  total_pages=0

  for page_num, image in enumerate(pdf_images):
    page_image_path = os.path.join(page_dir, f'page{page_num + 1}.jpg')
    image.save(page_image_path, 'JPEG')
    total_pages+=1

    image = cv2.imread(page_image_path)
    image = image[..., ::-1]
    layout = model.detect(image)

    print(table_images_dir)
    create_table_images(layout, image, table_images_dir,pdf_name,page_num)


In [ ]:
def extract_tables(pdf_folder, output_folder):
  pdf_files = os.listdir(pdf_folder)
  for pdf_file in pdf_files:
      if pdf_file.endswith('.pdf'):
          process_pdf(pdf_file, pdf_folder, output_folder)

In [ ]:
# Main function
def main():
  pdf_folder = '/content/drive/MyDrive/5th Year (2023)/AmritanshuKhareDualDegree2024/MTP_Dataset_todo'  # Replace with your PDFs folder path
  output_folder = '/content/drive/MyDrive/5th Year (2023)/AmritanshuKhareDualDegree2024/Test_Output_7'  # Replace with desired output folder path

  extract_tables(pdf_folder, output_folder)

if __name__ == "__main__":
  main()

j.mtener.2020.100483


In [ ]:
def extract_tables(pdf_folder, output_folder):
  pdf_files = os.listdir(pdf_folder)
  for pdf_file in pdf_files:
    if pdf_file.endswith('.pdf'):
        process_pdf(pdf_file, pdf_folder, output_folder)

def process_pdf(pdf_file, pdf_folder, output_folder):
  pdf_name = os.path.splitext(pdf_file)[0]
  pdf_path = os.path.join(pdf_folder, pdf_file)

  pdf_output_dir = os.path.join(output_folder, pdf_name)
  os.makedirs(pdf_output_dir, exist_ok=True)

  pdf_images = convert_from_path(pdf_path)
  page_dir = os.path.join(pdf_output_dir, pdf_name + '_pages')
  os.makedirs(page_dir, exist_ok=True)

  # ... (Rest of the processing code)

  table_images_dir = os.path.join(pdf_output_dir, 'table_imgs')
  create_table_images(layout, image, table_images_dir)
  process_table_images(table_images_dir, pdf_name, page_num)

def create_table_images(layout, image, table_images_dir):
    # ... (Your code to extract and save table images)

def process_table_images(table_images_dir, pdf_name, page_num):
  ocr = PaddleOCR(lang='en')
  table_images = [f for f in os.listdir(table_images_dir) if f.endswith('.jpg')]

  # ... (Rest of your code to process table images and create CSV)

# Main function
def main():
  pdf_folder = '/content/drive/MyDrive/Dataset_todo'  # Replace with your PDFs folder path
  output_folder = '/content/drive/MyDrive/Test_Output_6'  # Replace with desired output folder path

  extract_tables(pdf_folder, output_folder)

if __name__ == "__main__":
  main()
